In [2]:
!pip install tensorflow
!pip install nltk
!pip install snscrape

In [ ]:
import tensorflow as tf

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
cd /content/drive/MyDrive/IndexPage

In [ ]:
# new_model = tf.keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/ModelLSDM')
# new_model.summary()

In [ ]:
import pandas as pd 
import datetime
from tqdm.notebook import tqdm
import snscrape.modules.twitter as sntwitter

In [2]:
!pip install pyngrok
!pip install flask-ngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 761.3/761.3 kB 595.7 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-5.2.1-py3-none-any.whl size=19772 sha256=67d28bf2a2dafc1f4c0a1b3af6d5f4bf178ff2789c11db1e0e222bad2be73653
  Stored in directory: /Users/sandaludesilva/Library/Caches/pip/wheels/f6/89/59/49d4249e00957e94813ac136a335d10ed2e09a856c5096f95c
Successfully built pyngrok


In [ ]:
from pyngrok import ngrok
ngrok.set_auth_token('2NpJgVAPvnKxciFH5fjre1ZkRLl_6sr3HQirtTuoAciZZeoY3')

# Now you can use ngrok.connect() to expose a local web server


In [ ]:
#cd /content/drive/MyDrive/ModelLSTM

In [ ]:
import re

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup

In [ ]:
# removing not and no from stop words as we need them for our model training
stopwords = stopwords.words("english")
stopwords.remove('not')
stopwords.remove('no')

# intializing method for lemmatizing words
lemmatizer = WordNetLemmatizer()

# now creating funtion to clean our data
def cleaned_review(review):
    # remove any html tags
    new_review = BeautifulSoup(review).get_text()
    
    # remove urls from reviews
    no_urls = new_review.replace('http\S+', '').replace('www\S+', '')
    
    # remove any non-letters
    clean_review = re.sub("[^a-zA-Z]", " ", no_urls)
    
    # convert whole sentence to lowercase and split
    new_words = clean_review.lower().split()
    
    # converting stopwords list to set for faster search
    stops = set(stopwords)
    
    # using stopwords to remove irrelavent words and lemmatizing the final output
    final_words = [lemmatizer.lemmatize(word) for word in new_words if not word in stops]
    
    # return the final result
    return (" ".join(final_words))

In [ ]:
# Importing required libraries
from tensorflow.keras.models import load_model
import numpy as np

# Loading the saved model
newModel = load_model('ModelLSDM')
newModel.summary()



In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template, request

app = Flask(__name__)
run_with_ngrok(app)
import os

@app.route('/')
def index():

    return render_template("index.html")



In [ ]:
# input_text = "fddf"

In [ ]:
@app.route('/results', methods=['POST'])
def results():

    profile = request.form['username']
    print(profile)

    # Scrape Twitter profile
    now = datetime.datetime.now()
    twenty_four_hours_ago = now - datetime.timedelta(hours=24)
    formatted_time = twenty_four_hours_ago.strftime('%Y-%m-%d')
    userScraper = sntwitter.TwitterProfileScraper(profile)
    userdata = userScraper._get_entity()

    
    tweets = []
    input_text=""
    for i, tweet in enumerate(userScraper.get_items()):
        if isinstance(tweet, sntwitter.Tweet) and tweet.date.strftime('%Y-%m-%d') >= formatted_time:
            if(i > 1):
                break
            data = [tweet.date,
                    tweet.rawContent,
                    tweet.likeCount,
                    tweet.retweetCount]
            input_text += tweet.rawContent
            tweets.append(data)

    input_text = cleaned_review(input_text)

    # So we see above that max no.of words in each sentence is 40
    max_len = 40

    from tensorflow.keras.preprocessing.text import Tokenizer
    from tensorflow.keras.preprocessing.sequence import pad_sequences

    tokenizer = Tokenizer(num_words=10000)
    tokenizer.fit_on_texts(input_text)

    # # Preprocess input text
    input_sequence = tokenizer.texts_to_sequences([input_text])
    padded_input_sequence = pad_sequences(input_sequence, maxlen=max_len, padding='post')
    
    # Make prediction
    preds = newModel.predict(padded_input_sequence)
    sentiment_labels = ['Negative', 'Neutral', 'Positive']
    predicted_label = sentiment_labels[np.argmax(preds)]


    return render_template('results.html', predicted_label=predicted_label)


In [ ]:
app.run()